
*   Firstly I downlaoded the AA.csv from the zinc website and uploaded to the google drive

*   Then adjusted the shape of the array to fit the VAE. VAE model is referecened from Keras Website.


*   The model was iterated over various no of samples ranging from 100 to 400000. But the model was not behaving in a good way. Also, the parameters were varied for various smaple data to fetch for a good result.

*   After many iterations, the sample data was fixed to 500 and the model below.
Then tried for validating the result for 10000 predicted smile representaion.


*   But the latent space in which I was searching was unable to produce a valid smile representation. This doesn't mean that a valid smile representation will not exist.








In [ ]:
#Installing RDKit in Collab
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3669    0  3669    0     0  18253      0 --:--:-- --:--:-- --:--:-- 18253
100 20.2M  100 20.2M    0     0  5244k      0  0:00:03  0:00:03 --:--:-- 6906k
mv: cannot move 'lib/python3.6/site-packages/rdkit' to '/usr/local/lib/python3.6/dist-packages/rdkit': Directory not empty
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libboost_python3.so.1.65.1': File exists


In [ ]:
#mounting google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

drive/  lib/  sample_data/


In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
ls

 AA.csv
'Assignment 3.pdf'
'Atomic Giants.mov'
 CelebA@
'CelebA (1)'/
 CelebA_img/
'Colab Notebooks'/
 decoder.pkl
 drug_discovery@
'GC Health - Wearable Tech.gdoc'
 generated_plot_e1000.png
 generated_plot_e140.png
 generated_plot_e150.png
 generated_plot_e520.png
 generated_plot_e530.png
 generated_plot_e540.png
 generated_plot_e550.png
 generated_plot_e560.png
 generated_plot_e570.png
 generated_plot_e580.png
 generated_plot_e590.png
 generated_plot_e600.png
 generated_plot_e610.png
 generated_plot_e620.png
 generated_plot_e630.png
 generated_plot_e640.png
 generated_plot_e650.png
 generated_plot_e660.png
 generated_plot_e670.png
 generated_plot_e680.png
 generated_plot_e690.png
 generated_plot_e700.png
 generated_plot_e710.png
 generated_plot_e720.png
 generated_plot_e730.png
 generated_plot_e740.png
 generated_plot_e750.png
 generated_plot_e760.png
 generated_plot_e770.png
 generated_plot_e780.png
 generated_plot_e790.png
 generated_plot_e800.png
 generated_plot_e810.png
 generate

In [ ]:
#taking AA.csv from 'http://zinc.docking.org/substances/subsets/AA/'
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/drug_discovery/AA.csv')
data

,zinc_id,smiles
0,ZINC000000008151,C[C@H]1[C@@H](O)[C@H](CO)O[C@@H](O)[C@@H]1N
1,ZINC000000008153,CC[C@@H]1[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
2,ZINC000000008155,CC1(C)[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
3,ZINC000000018276,CS[C@@H]1CN[C@@H](CO)[C@H](O)[C@H]1O
4,ZINC000000018279,CS[C@@H]1[C@@H](O)CN[C@@H](CO)[C@@H]1O
...,...,...
427851,ZINC000242463989,O[Cl+3](O)(O)O
427852,ZINC000247713634,O1[SiH2][SiH2]O[SiH2][SiH2]1
427853,ZINC000252581626,O[Si](O)(O)F
427854,ZINC000685945533,Cn1nnnc1S(=O)(=O)F


In [ ]:
data['smiles'].apply(len).max()#checking for max length of smiles

62

In [ ]:
data['smiles'].apply(len).min()#checking for min length of smiles

4

In [ ]:
from rdkit import Chem #importing rdkit

In [ ]:
#Definining encoder for smile encoding to array and from arry to smile representation
SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder( smiles, maxlen=62 ):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles( smiles ))
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c]] = 1
    return X
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[i]
    return smi

In [ ]:
#initialising the data. Taking 500 smiles representation
#This was not performing good on more than 1000 smiles.
import numpy as np
x = []
for i in data['smiles'][:500]:
    #print(smiles_encoder(i))
    x.append(smiles_encoder(i))
arr = np.array(x)

In [ ]:
arr.shape#checking for shape

(500, 62, 56)

In [ ]:
arr = arr.reshape(-1,62,56,1) #reshaping the array to (500,62,56,1) for VAE

In [ ]:
arr.shape#rechecking the shape 

(500, 62, 56, 1)

In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches 
%matplotlib inline
from scipy.stats import norm 
import keras
from keras import layers
from keras.models import Model 
from keras import metrics
from keras import backend as K 
K.clear_session()
np.random.seed(237)
import scipy.stats as st
import tensorflow.compat.v1.keras.backend as K 
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers import Adam 
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np
from scipy.stats import multivariate_normal

In [ ]:
#Encoder for VAE
inp_shape = (62, 56,1)
batch_size = 16
latent_dim = 2
inp_arr = keras.Input(shape=inp_shape)

x = layers.Conv1D(32, 7,
                  padding='same',
                  activation='relu',
                  )(inp_arr)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu',
                  strides=(2, 2))(x)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu')(x)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
z_mu = layers.Dense(latent_dim)(x)
z_log_sigma = layers.Dense(latent_dim)(x)

In [ ]:
#Defining the sampling function
def sampling(args):
  z_mu, z_log_sigma = args
  epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
  mean=0., stddev=1.)
  return z_mu + K.exp(z_log_sigma) * epsilon
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

In [ ]:
#Defining the decoder
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
                           padding='same',
                           activation='relu',
                           strides=(2, 2))(x)
x = layers.Conv2D(1, 3,
                  padding='same',
                  activation='sigmoid')(x)
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [ ]:
#Loss
class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded) #Recin loss
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1) #KL loss
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([inp_arr, z_decoded])

In [ ]:
# VAE model statement
vae = Model(inp_arr, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 62, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 62, 56, 32)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 31, 28, 128)  36992       conv1d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 31, 28, 128)  147584      conv2d[0][0]                     
____________________________________________________________________________________________

In [ ]:
vae.fit(x=arr, y=None,
        shuffle=False,
        epochs=120,
        batch_size=batch_size,
        )

Train on 500 samples
Epoch 1/120
500/500 [==============================] - 1s 1ms/sample - loss: 0.2035
Epoch 2/120
500/500 [==============================] - 0s 417us/sample - loss: 0.0488
Epoch 3/120
500/500 [==============================] - 0s 409us/sample - loss: 0.0404
Epoch 4/120
500/500 [==============================] - 0s 416us/sample - loss: 0.0375
Epoch 5/120
500/500 [==============================] - 0s 407us/sample - loss: 0.0357
Epoch 6/120
500/500 [==============================] - 0s 412us/sample - loss: 0.0349
Epoch 7/120
500/500 [==============================] - 0s 414us/sample - loss: 0.0343
Epoch 8/120
500/500 [==============================] - 0s 403us/sample - loss: 0.0341
Epoch 9/120
500/500 [==============================] - 0s 411us/sample - loss: 0.0333
Epoch 10/120
500/500 [==============================] - 0s 412us/sample - loss: 0.0323
Epoch 11/120
500/500 [==============================] - 0s 409us/sample - loss: 0.0313
Epoch 12/120
500/500 [===========

In [51]:
#Prinitng the 10000 random predicted smiles from VAE
import random
result = []
for i in random.sample(range(1, 10000000), 100):
  for j in random.sample(range(1, 10000000), 100):
    sample_vector = np.array([[i,j]])
    res = decoder.predict(sample_vector)
    res = res[0].reshape(62,56)
    result.append(smiles_decoder(res).replace(' ',''))
    # print(i,j)
set(result)

{'1(/N/)N0',
 '1(NN/)N0',
 '1-N//(-))N#M00',
 '1-N//(-))N#oM00',
 '1-N//(-)1N#oM00',
 '1-N//(/))N#M00',
 '1-N//(/)N#M0',
 '1-N//(/)N#M00',
 '1C(NN/)N0',
 '1C(NN/)NC0',
 '1N/(/)N1#M0',
 '1N/(/)N1M0',
 '1N/(/)NM0',
 '1N//(/)N#M0',
 '1N//(/)N1#M0',
 '1N//)N0',
 '1N//)NM0',
 '1N/2/)NM0',
 '1N/N/)N0',
 'C1C(NCN/)NC0',
 'C1C(NN/)NC0',
 'C1C(NNCN/)NC0',
 'C1C(NNCN/)NC@H](OF',
 'C1C(NNCN/)NC@](O',
 'C1C(NNCN/)NC@](O0',
 'C1C(NNCN/)NC@](OF',
 'C1C(NNCN/)NCO0',
 'C1C(NNCN/)NC](O0',
 'C1C(NNCN/)NC]O0',
 'C=C(@CN1O[CC@H](O)@',
 'C=C(@CN1O[[C@H](O)',
 'C=C(@CN1O[[C@H](O)@',
 'C=C(C@CN1O[CC@H](O)@',
 'C=C(C@N1O[CC@H](O)@',
 'C=C(CN1O[[C@H](O)',
 'C=C(NNCN/)NC@H](OF',
 'C=C(NNCN/1)NC@H](OF',
 'C=C(NNCN/1)N[C@H](OF',
 'C=C(NNCN/1)[[C@H](OF',
 'C=C(NNCN/1O[[C@H](OF',
 'C=C(NOCN/1O[[C@H](O)',
 'C=C(NOCN/1O[[C@H](OF',
 'C=C(OCN/1O[[C@H](O)',
 'C=C(OCN1O[[C@H](O)',
 'C=C-C@@H1O[C@@H](O)C@H]([C@H]()[C@]',
 'C=C-C@@H1O[C@@H](O)C@H]([C@H]()[C@]]',
 'C=C-C@@H]1O[C@@H](O)C@H]([C@H]()[C@]]',
 'C=C-C@@H]1O[C@@H]

In [41]:
from rdkit.Chem import Draw
for i in result:
  try:
    m = Chem.MolFromSmiles(smiles_decoder(i))
    fig = Draw.MolToMPL(m, size=(200,200))
    print('possible')
  except:
    continue

#Checking the predcited smile representation (10000) whether it is a valid representation or not. Was unable to find one. But this was for the latent spave I was seeking into. There may be other latent space where a valid smile representation may exist.

In [52]:
result[2098]

'C=C-C@@H]1O[C@@H](O)[C@H]([C@H]()[C@]]'

In [55]:
result[1099]

'C=C[C@@H1O[C@@H](O)C[C@H]()[C]'

In [57]:
result[675]

'C1C(NNCN/)NC0'